# Brazilian Economics and Demographics Data Model
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

[Brazil Cities](https://www.kaggle.com/crisparada/brazilian-cities)

[Enem 2014](http://dados.gov.br/dataset/microdados-do-exame-nacional-do-ensino-medio-enem/resource/94731b73-e9f1-4262-b8d6-479b6d02a6f0?inner_span=True)

This dataset is a compilation of several publicly available information about Brazilian Municipalities.

- The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# # Packages needed
# !pip3 install psycopg2
# !pip3 install pyspark
# !pip3 install pandas
# !pip3 install xlrd

In [2]:
# Do all imports and installs here
import re
import os
import psycopg2
# import requests
import pandas as pd
from etl import process_cities_data, process_enem_data, create_spark_session

from datetime import datetime, timedelta

brazil_cities_path = os.getcwd() + '/dataBrazil/brazil_cities.csv'
brazil_cities_dictionary_path = os.getcwd() + '/dataBrazil/brazil_cities_dictionary.csv'
enem_2018_path = os.getcwd() + '/dataBrazil/enem/enem_2018.csv'
enem_2018_dictionary_path = os.getcwd() + '/dataBrazil/enem/enem_2018_dictionary.xlsx'


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# ## Uncomment to read real dataset
# enem2018_df = pd.read_csv(enem_2018_path, delimiter=";")
# enem2018_df.fillna(0, inplace=True)
# print(enem2018_df.shape)

(5513747, 41)


In [ ]:
# ## Uncomment to create dataset test 
# enem_test_path = os.getcwd() + '/dataBrazil/enem/enem_test.csv'
# enem_test = enem2018_df.head(10000)
# enem_test.to_csv(enem_test_path, sep=';')
# print(enem_test.shape)
# enem_test.head()

In [12]:
# Comment when use real dataset. This dataset is only for test
enem_2018_path = os.getcwd() + '/dataBrazil/enem/enem_test.csv'
enem2018_df = pd.read_csv(enem_2018_path, delimiter=";")
enem2018_df.fillna(0, inplace=True)
# del enem2018_df['Unnamed: 0']
print(enem2018_df.shape)
enem2018_df.head()

(10000, 41)


,NU_INSCRICAO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,...,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,180008202043,5300108,Brasília,53,DF,44.0,M,1.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,180007197856,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,...,1.0,1.0,1.0,1.0,521.0,568.7,513.1,480.3,1.0,280.0
2,180008517434,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,180007661228,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,180008787987,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,...,0.0,1.0,1.0,0.0,0.0,419.9,345.8,0.0,1.0,360.0


In [13]:
# Read in the data here
brazil_df = pd.read_csv(brazil_cities_path, delimiter=";")
brazil_df.fillna(0, inplace=True)
print(brazil_df.shape)
brazil_df.head()

(5573, 81)


,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,Pu_Bank,Pr_Assets,Pu_Assets,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,0.0,0.0,0.0,2158.0,1246.0,0.0,0.0,0.0,0.0,1.0
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,0.0,0.0,0.0,2227.0,1142.0,0.0,0.0,0.0,0.0,1.0
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,1.0,33724584.0,67091904.0,2838.0,1426.0,0.0,0.0,0.0,0.0,3.0
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2.0,44974716.0,371922572.0,6928.0,2953.0,0.0,0.0,0.0,0.0,4.0
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,4.0,76181384.0,800078483.0,5277.0,25661.0,0.0,0.0,0.0,0.0,2.0


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### Creating Spark Session

In [14]:
spark = create_spark_session()

#### ETL Process and Parquet Creation

In [ ]:
outputS3 = ''

process_cities_data(spark, brazil_cities_path, output_data=outputS3)

process_enem_data(spark, enem_2018_path, output_data)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.